## 데이터 전처리

### 필요 칼럼만 남기고 두 데이터 이름 맞추기

In [1]:
import pandas as pd
import numpy as np
ten = pd.read_csv('관세청_HS부호_20250101.csv',dtype={'HS부호': str})
two = pd.read_csv('hscode_raw_2_4_6.csv', dtype={'hscode': str, 'parent': str})

display(ten.head(), two.head())

,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,한국표준무역분류명,수량단위최대단가,중량단위최대단가,수량단위코드,중량단위코드,수출성질코드,수입성질코드,품목규격명,필수규격명,참고규격명,규격설명,규격사항내용,성질통합분류코드,성질통합분류코드명
0,0101211000,20130101,20251231,농가 사육용,For farm breeding,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020101.0,(말)
1,0101219000,20120101,20251231,기타,Other,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020101.0,(말)
2,0101291000,20120101,20251231,경주말,Horses for racing,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020101.0,(말)
3,0101299000,20120101,20251231,기타,Other,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020101.0,(말)
4,0101300000,20220101,20251231,당나귀,Asses,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020190.0,(기타 산 동물)


,section,hscode,description,parent,level
0,I,01,Animals; live,TOTAL,2
1,I,0101,"Horses, asses, mules and hinnies; live",01,4
2,I,010121,"Horses; live, pure-bred breeding animals",0101,6
3,I,010129,"Horses; live, other than pure-bred breeding an...",0101,6
4,I,010130,Asses; live,0101,6


In [2]:
display(ten.columns, two.columns)

Index(['HS부호', '적용시작일자', '적용종료일자', '한글품목명', '영문품목명', '한국표준무역분류명', '수량단위최대단가',
       '중량단위최대단가', '수량단위코드', '중량단위코드', '수출성질코드', '수입성질코드', '품목규격명', '필수규격명',
       '참고규격명', '규격설명', '규격사항내용', '성질통합분류코드', '성질통합분류코드명'],
      dtype='object')

Index(['section', 'hscode', 'description', 'parent', 'level'], dtype='object')

In [3]:
ten = ten[['HS부호', '한글품목명', '영문품목명', '성질통합분류코드명']]
ten.head()


,HS부호,한글품목명,영문품목명,성질통합분류코드명
0,0101211000,농가 사육용,For farm breeding,(말)
1,0101219000,기타,Other,(말)
2,0101291000,경주말,Horses for racing,(말)
3,0101299000,기타,Other,(말)
4,0101300000,당나귀,Asses,(기타 산 동물)


In [4]:
df_detail = ten.rename(columns={'HS부호': 'code', '한글품목명': 'desc_ko', '영문품목명': 'desc_en', 
                                'parent_code': 'parent_code', '성질통합분류코드명': 'category'})
df_structure = two.rename(columns={'hscode': 'code', 'description': 'desc_en', 'parent': 'parent_code'})
display(df_detail.head(), df_structure.head())

,code,desc_ko,desc_en,category
0,0101211000,농가 사육용,For farm breeding,(말)
1,0101219000,기타,Other,(말)
2,0101291000,경주말,Horses for racing,(말)
3,0101299000,기타,Other,(말)
4,0101300000,당나귀,Asses,(기타 산 동물)


,section,code,desc_en,parent_code,level
0,I,01,Animals; live,TOTAL,2
1,I,0101,"Horses, asses, mules and hinnies; live",01,4
2,I,010121,"Horses; live, pure-bred breeding animals",0101,6
3,I,010129,"Horses; live, other than pure-bred breeding an...",0101,6
4,I,010130,Asses; live,0101,6


In [5]:
df_detail['parent_code'] = df_detail['code'].str[:6]
df_detail.head()

,code,desc_ko,desc_en,category,parent_code
0,0101211000,농가 사육용,For farm breeding,(말),010121
1,0101219000,기타,Other,(말),010121
2,0101291000,경주말,Horses for racing,(말),010129
3,0101299000,기타,Other,(말),010129
4,0101300000,당나귀,Asses,(기타 산 동물),010130


In [6]:
# 1. 상세 데이터 (10자리)
df_detail_clean = df_detail.copy()
df_detail_clean['description'] = df_detail_clean['desc_ko'].fillna('') + ' (' + df_detail_clean['desc_en'].fillna('') + ')'
df_detail_clean = df_detail_clean[['code', 'description', 'parent_code', 'category']].copy()
df_detail_clean['source'] = 'DETAIL'

# 2. 구조 데이터 (2, 4, 6자리)
df_structure_clean = df_structure.copy()
# 구조 데이터는 영문 description만 있으므로 그대로 사용
df_structure_clean['description'] = df_structure_clean['desc_en'].fillna('')
df_structure_clean = df_structure_clean[['code', 'description', 'parent_code']].copy()
df_structure_clean['source'] = 'STRUCTURE'

display(df_detail_clean.head(10), df_structure_clean.head(10))

,code,description,parent_code,category,source
0,0101211000,농가 사육용 (For farm breeding),010121,(말),DETAIL
1,0101219000,기타 (Other),010121,(말),DETAIL
2,0101291000,경주말 (Horses for racing),010129,(말),DETAIL
3,0101299000,기타 (Other),010129,(말),DETAIL
4,0101300000,당나귀 (Asses),010130,(기타 산 동물),DETAIL
5,0101900000,기타 (Other),010190,(기타 산 동물),DETAIL
6,0102211000,젖소 (For milk ),010221,(소),DETAIL
7,0102212000,육우(肉牛) (For meat),010221,(소),DETAIL
8,0102219000,기타 (Other),010221,(소),DETAIL
9,0102291000,젖소 (For milk ),010229,(소),DETAIL


,code,description,parent_code,source
0,01,Animals; live,TOTAL,STRUCTURE
1,0101,"Horses, asses, mules and hinnies; live",01,STRUCTURE
2,010121,"Horses; live, pure-bred breeding animals",0101,STRUCTURE
3,010129,"Horses; live, other than pure-bred breeding an...",0101,STRUCTURE
4,010130,Asses; live,0101,STRUCTURE
5,010190,Mules and hinnies; live,0101,STRUCTURE
6,0102,Bovine animals; live,01,STRUCTURE
7,010221,"Cattle; live, pure-bred breeding animals",0102,STRUCTURE
8,010229,"Cattle; live, other than pure-bred breeding an...",0102,STRUCTURE
9,010231,"Buffalo; live, pure-bred breeding animals",0102,STRUCTURE


## 키워드 추출
### 구조 데이터 먼저

In [7]:
# 데이터 괄호, 배제 언어 확인

# ⚠️ df_structure_clean이 현재 환경에 정의되어 있다고 가정합니다.

if 'description' in df_structure_clean.columns:
    descriptions = df_structure_clean['description'].fillna('')
    total_descriptions = len(descriptions)

    # 1. 배제/포괄 구문 패턴 정의 (체계적인 분석을 위해)
    EXCLUSION_PATTERNS = {
        "other than": r'other than',
        ";": r';',
        "excluding": r'excluding',
        "etc (포괄)": r'etc[\.]?', 
        "n.e.s / n.e.c": r'n\.e\.s|n\.e\.c' 
    }

    # 2. 괄호 패턴 정의
    BRACKET_PATTERNS = {
        "Round Brackets ()": r'\(|\)',
        "Square Brackets []": r'\[|\]'
    }

    print("--- HS CODE DESCRIPTION 패턴 분석 (수정 없음) ---")
    print(f"총 분석 대상 항목: {total_descriptions}개\n")

    # --- 1. 배제/포괄 구문 분석 ---
    print("--- 1. 배제/포괄 구문 포함 여부 ---")
    for key, pattern in EXCLUSION_PATTERNS.items():
        # 대소문자 무시 (case=False)하여 count
        count = descriptions.str.contains(pattern, case=False, na=False).sum()
        percentage = (count / total_descriptions) * 100
        print(f"'{key}' 포함: {count}개 ({percentage:.2f}%)")

    # --- 2. 괄호 사용 분석 ---
    print("\n--- 2. 괄호 문자 사용 여부 ---")
    for key, pattern in BRACKET_PATTERNS.items():
        count = descriptions.str.contains(pattern, case=False, na=False).sum()
        percentage = (count / total_descriptions) * 100
        print(f"'{key}' 포함: {count}개 ({percentage:.2f}%)")
        
else:
    print("❌ 오류: 'df_structure_clean' 데이터프레임에 'description' 컬럼이 정의되어 있지 않습니다.")

--- HS CODE DESCRIPTION 패턴 분석 (수정 없음) ---
총 분석 대상 항목: 6940개

--- 1. 배제/포괄 구문 포함 여부 ---
'other than' 포함: 1069개 (15.40%)
';' 포함: 6141개 (88.49%)
'excluding' 포함: 539개 (7.77%)
'etc (포괄)' 포함: 41개 (0.59%)
'n.e.s / n.e.c' 포함: 840개 (12.10%)

--- 2. 괄호 문자 사용 여부 ---
'Round Brackets ()' 포함: 2252개 (32.45%)
'Square Brackets []' 포함: 3개 (0.04%)


저런 것들이 너무 많아서 명사 위주로 뽑기로 결정

In [45]:
#import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  ㅇ


Command 'ㅇ' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  d



Download which package (l=list; x=cancel)?


...

  Identifier>  averaged_perceptron_tagger


        /root/nltk_data...
      Package averaged_perceptron_tagger is already up-to-date!



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  q


True

In [8]:
import pandas as pd
import re

def clean_text_for_pos(text: str) -> str:
    """
    세미콜론 전략(핵심 상품 정의 추출) 및 괄호 제거를 수행하여
    NLTK 품사 태깅이 용이한 텍스트로 정화합니다.
    """
    if pd.isna(text) or text is None:
        return ""
    
    text_str = str(text).strip()
    
    # --- 1. 세미콜론 전략: 상품 본질만 추출 ---
    # 세미콜론(;)이 있다면 앞부분만 사용 (보조 조건 제거)
    if ';' in text_str:
        text_str = text_str.split(';')[0].strip()
    
    # --- 2. 괄호 문자 제거 (내용 유지 요청 반영) ---
    # 괄호 문자만 제거하여 괄호 안의 내용이 명사로 추출되도록 합니다.
    text_str = text_str.replace('(', ' ').replace(')', ' ').strip()
    
    # --- 3. 배제 구문 제거 (Other than / Excluding / Except) ---
    # 세미콜론 전략을 사용했더라도 혹시 남아있을 수 있는 배제 구문은 제거합니다.
    text_str = re.sub(r'other than.*', '', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'excluding.*', '', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'except.*', '', text_str, flags=re.IGNORECASE)
    
    # --- 4. 잔여물 및 구두점 정리 (단어 분리 유도) ---
    # 쉼표, 마침표, 하이픈 등 모든 구두점을 공백으로 대체합니다.
    text_str = re.sub(r'[,\.;:()\-\[\]]', ' ', text_str) 
    
    # n.e.s., etc. 등의 약어는 제거합니다.
    text_str = re.sub(r'n\s*e\s*s|n\s*e\s*c|etc[\.]?', '', text_str, flags=re.IGNORECASE)

    # 수량, 측정, 단위 정보 제거 (185g 등)
    text_str = re.sub(r'\d+[a-z]{1,2}', '', text_str, flags=re.IGNORECASE)
    text_str = re.sub(r'not more than|exceeding', '', text_str, flags=re.IGNORECASE)

    # 다중 공백을 단일 공백으로 변환하고 최종 정리
    text_str = re.sub(r'\s+', ' ', text_str).strip()
    
    return text_str

# --- 다음 셀에 이어서 실행하세요 ---

In [9]:
import nltk
from typing import List

# NLTK 데이터가 다운로드 되었다고 가정합니다.

def extract_pos_keywords(description: str) -> List[str]:
    """
    NLTK 품사 태깅을 사용하여 명사(NN) 및 형용사(JJ)만 추출합니다.
    (obtained와 같은 동사나 기능성 단어 자동 제거)
    """
    if not description:
        return []
        
    # 1. 토큰화 및 품사 태깅
    words = description.lower().split()
    tagged_words = nltk.tag.pos_tag(words)
    
    # 2. 핵심 품사 필터링: 명사(NN*)와 형용사(JJ*)만 허용
    allowed_tags = {'NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS'}
    
    keywords = [
        word for word, tag in tagged_words 
        if tag in allowed_tags and len(word) > 2 and word.isalpha()
    ]
    
    # 3. HS 코드 도메인 잔여 불용어 필터링 (POS Tagging으로 걸러지지 않는 명사/형용사)
    # (예: 분류의 주체가 아닌, 범용적인 속성/용어)
    hs_stop_words = {
        'parts', 'pieces', 'thereof', 'subheading', 'chapter', 'heading',
        'level', 'mixtures', 'whether', 'known', 'having', 'other', 
        'used', 'products', 'articles', 'produce', 'raw', 'made', 
        'containing', 'total', 'certain', 'various', 'net', 'edible',
        'goods', 'live', 'fresh', 'frozen', 'dried', 'animals'
    }
    
    filtered_keywords = [
        word for word in keywords if word not in hs_stop_words
    ]

    return list(set(filtered_keywords))

# --- 다음 셀에 이어서 실행하세요 ---

In [10]:
# ⚠️ df_structure_clean이 정의되어 있다고 가정합니다.
# ⚠️ code, description, parent_code 컬럼이 있어야 합니다.

def create_structure_category(df_structure_clean: pd.DataFrame) -> pd.DataFrame:
    """
    df_structure_clean에 Category 컬럼을 생성하고, 
    '기타' 항목에 상위 코드 정보를 상속합니다.
    """
    
    # 1. description 클리닝 및 초기 키워드 추출
    df = df_structure_clean.copy()
    # clean_text_for_pos 함수를 적용하여 텍스트를 정화합니다.
    df['description_clean'] = df['description'].apply(clean_text_for_pos)
    
    # 정화된 텍스트에서 POS 태깅 기반 키워드 추출
    df['category'] = df['description_clean'].apply(extract_pos_keywords)
    
    # 2. 데이터 정렬 (상위 코드가 먼저 오도록)
    df = df.sort_values(by='code', ascending=True).reset_index(drop=True)
    
    # 3. '기타' 항목에 상위 분류 정보 상속
    
    # 상위 코드의 깨끗한 설명(description_clean)을 미리 딕셔너리에 저장
    code_desc_map = df.set_index('code')['description_clean'].to_dict()
    
    for index, row in df.iterrows():
        # '기타'나 키워드가 부족한 항목을 상속 대상으로 지정
        is_other = 'other' in row['description_clean'].lower() or not row['category']
        
        if is_other:
            parent_code = row['parent_code']
            parent_desc = code_desc_map.get(parent_code, '')
            
            if parent_desc:
                # 상위 설명에서 키워드 추출 후 현재 '기타' 항목에 상속/병합
                parent_keywords = extract_pos_keywords(parent_desc)
                current_keywords = set(row['category'])
                current_keywords.update(parent_keywords)
                
                df.at[index, 'category'] = list(current_keywords)

        # 최종 Category는 텍스트 형태로 저장
        df.at[index, 'category_str'] = ", ".join(df.at[index, 'category'])
        
    return df

# --- 최종 실행 ---

df_final_structure_categorized = create_structure_category(df_structure_clean)
display(df_final_structure_categorized.head(10))

,code,description,parent_code,source,description_clean,category,category_str
0,01,Animals; live,TOTAL,STRUCTURE,Animals,[commodities],commodities
1,0101,"Horses, asses, mules and hinnies; live",01,STRUCTURE,Horses asses mules and hinnies,"[mules, hinnies, horses]","mules, hinnies, horses"
2,010121,"Horses; live, pure-bred breeding animals",0101,STRUCTURE,Horses,[horses],horses
3,010129,"Horses; live, other than pure-bred breeding an...",0101,STRUCTURE,Horses,[horses],horses
4,010130,Asses; live,0101,STRUCTURE,Asses,[asses],asses
5,010190,Mules and hinnies; live,0101,STRUCTURE,Mules and hinnies,"[hinnies, mules]","hinnies, mules"
6,0102,Bovine animals; live,01,STRUCTURE,Bovine animals,[bovine],bovine
7,010221,"Cattle; live, pure-bred breeding animals",0102,STRUCTURE,Cattle,[cattle],cattle
8,010229,"Cattle; live, other than pure-bred breeding an...",0102,STRUCTURE,Cattle,[cattle],cattle
9,010231,"Buffalo; live, pure-bred breeding animals",0102,STRUCTURE,Buffalo,[buffalo],buffalo


In [11]:
# 잔잔한 전처리
df_structure_final = df_final_structure_categorized[['code','description', 'parent_code', 'source','category_str']]
df_structure_final = df_structure_final.rename(columns={'category_str': 'category'})
df_structure_final.loc[df_structure_final['parent_code'] == 'TOTAL', 'parent_code'] = np.nan
df_structure_final.head()

,code,description,parent_code,source,category
0,01,Animals; live,NaN,STRUCTURE,commodities
1,0101,"Horses, asses, mules and hinnies; live",01,STRUCTURE,"mules, hinnies, horses"
2,010121,"Horses; live, pure-bred breeding animals",0101,STRUCTURE,horses
3,010129,"Horses; live, other than pure-bred breeding an...",0101,STRUCTURE,horses
4,010130,Asses; live,0101,STRUCTURE,asses


### 10개짜리 전처리 및 키워드 뽑기

In [12]:
df_detail_clean.isna().sum()

code              0
description       0
parent_code       0
category       1140
source            0
dtype: int64

In [13]:
df_detail_clean.head(10)

,code,description,parent_code,category,source
0,0101211000,농가 사육용 (For farm breeding),010121,(말),DETAIL
1,0101219000,기타 (Other),010121,(말),DETAIL
2,0101291000,경주말 (Horses for racing),010129,(말),DETAIL
3,0101299000,기타 (Other),010129,(말),DETAIL
4,0101300000,당나귀 (Asses),010130,(기타 산 동물),DETAIL
5,0101900000,기타 (Other),010190,(기타 산 동물),DETAIL
6,0102211000,젖소 (For milk ),010221,(소),DETAIL
7,0102212000,육우(肉牛) (For meat),010221,(소),DETAIL
8,0102219000,기타 (Other),010221,(소),DETAIL
9,0102291000,젖소 (For milk ),010229,(소),DETAIL


In [14]:
import pandas as pd

def remove_brackets_only(text):
    # 1. NaN 또는 비문자열 타입 체크: TypeError 방지
    if pd.isna(text) or not isinstance(text, str):
        return text
    
    # 2. 열린 괄호 '('만 제거
    result = text.replace('(', '')
    
    # 3. 닫힌 괄호 ')'만 제거
    result = result.replace(')', '')
    
    # 4. 앞뒤 공백 정리
    return result.strip()

# --- 적용 예시 ---
# 'category' 컬럼에 적용
df_detail_clean['category'] = df_detail_clean['category'].apply(remove_brackets_only)

df_detail_clean.head()

,code,description,parent_code,category,source
0,0101211000,농가 사육용 (For farm breeding),010121,말,DETAIL
1,0101219000,기타 (Other),010121,말,DETAIL
2,0101291000,경주말 (Horses for racing),010129,말,DETAIL
3,0101299000,기타 (Other),010129,말,DETAIL
4,0101300000,당나귀 (Asses),010130,기타 산 동물,DETAIL


In [15]:
# 두 데이터셋 합치고 구조 데이터 우선으로 중복 제거
df_master = pd.concat([df_structure_final, df_detail_clean], ignore_index=True)
df_master = df_master.drop_duplicates(subset=['code'], keep='first')
df_master = df_master[df_master['code'] != 'nan'].reset_index(drop=True)
df_master.sort_values(by='code', inplace=True)
df_master.head()

,code,description,parent_code,source,category
0,01,Animals; live,NaN,STRUCTURE,commodities
1,0101,"Horses, asses, mules and hinnies; live",01,STRUCTURE,"mules, hinnies, horses"
2,010121,"Horses; live, pure-bred breeding animals",0101,STRUCTURE,horses
6940,0101211000,농가 사육용 (For farm breeding),010121,DETAIL,말
6941,0101219000,기타 (Other),010121,DETAIL,말


In [16]:
import numpy as np

# 'category' 컬럼을 안전하게 비어있는지 확인하는 함수
def is_category_empty_final(value):
    # 1. NaN이거나 빈 문자열인 경우
    if pd.isna(value):
        return True
    return False


# 비어있는 Category 행만 추출
df_empty_categories = df_detail_clean[
    df_detail_clean['category'].apply(is_category_empty_final)
].copy()

# 결과 출력
display(df_empty_categories.shape)

(1140, 5)

In [17]:
df_master

,code,description,parent_code,source,category
0,01,Animals; live,NaN,STRUCTURE,commodities
1,0101,"Horses, asses, mules and hinnies; live",01,STRUCTURE,"mules, hinnies, horses"
2,010121,"Horses; live, pure-bred breeding animals",0101,STRUCTURE,horses
6940,0101211000,농가 사육용 (For farm breeding),010121,DETAIL,말
6941,0101219000,기타 (Other),010121,DETAIL,말
...,...,...,...,...,...
19406,9706903000,기타 (Other),970690,DETAIL,수집품 및 골동품
6936,99,Commodities not specified according to kind,NaN,STRUCTURE,"kind, commodities"
6937,9999,Commodities not specified according to kind,99,STRUCTURE,"kind, commodities"
6938,999999,Commodities not specified according to kind,9999,STRUCTURE,"kind, commodities"


In [127]:
# pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.1 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'langdetect' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'langdetect'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=3ffc6a7b43ca9cf8129adc7cf1479db3411ec6d2579d59c8911eb3191340686a
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
import re
from langdetect import detect, DetectorFactory
from typing import List

# langdetect의 비결정성을 줄이기 위해 시드 설정
DetectorFactory.seed = 42 

def extract_state_keywords(description: str) -> List[str]:
    """
    텍스트에서 'dried', 'smoked', 'frozen' 등 분류에 중요한 상태 및 처리 방법을 나타내는 
    명사/형용사 키워드를 추출합니다.
    """
    if not description: return []
    
    words = description.lower().split()
    
    # POS Tagging을 사용하지 않고, 상태/처리 관련 키워드 딕셔너리로 추출
    state_terms = {
        'dried', 'smoked', 'frozen', 'chilled', 'salted', 'preserved', 'pickled', 
        'cooked', 'fresh', 'live', 'prepared', 'uncut', 'cut'
    }
    
    # description에 포함된 상태/처리 용어를 추출
    extracted_terms = [word for word in words if word in state_terms]
    
    return list(set(extracted_terms))

def clean_text_for_pos(text: str) -> str:
    """
    1. 가장 바깥쪽 괄호 안의 영문 부분만 추출합니다.
    2. 모든 한글 문자(Hangul)를 제거하여 NLTK에 순수 영문만 전달합니다.
    """
    if pd.isna(text) or text is None: return ""
    text_str = str(text).strip()
    
    # 1. 영문 부분만 추출 (가장 바깥쪽 괄호 안의 내용만 사용)
    # [가루와 웨이스트(waste)를 포함한다] (Young antlers a...) 형식에서 영문만 추출
    english_part = text_str
    
    # 텍스트 끝에 있는 괄호 쌍 (가장 바깥쪽 쌍)을 찾습니다.
    match = re.search(r'\((.*?)\)$', text_str)
    if match: 
        english_part = match.group(1).strip()
    
    # 2. 한글 문자 (Hangul) 제거 (핵심 로직)
    # ASCII 영문, 숫자, 공백, 기본 구두점(,-)을 제외한 모든 유니코드 문자(한글)를 제거합니다.
    # [^A-Za-z0-9\s,\-&] 패턴을 사용하여 한글을 확실히 제거
    cleaned_english = re.sub(r'[^A-Za-z0-9\s,\-&]', ' ', english_part).strip()

    # 3. 배제 구문 제거 (Other than / Excluding / Except)
    cleaned_english = re.sub(r'other than.*|excluding.*|except.*', '', cleaned_english, flags=re.IGNORECASE)

    # 4. 잔여물 및 구두점 정리
    cleaned_english = re.sub(r'n\s*e\s*s|n\s*e\s*c|etc[\.]?', '', cleaned_english, flags=re.IGNORECASE)
    cleaned_english = re.sub(r'[,\.;:()\-\[\]]', ' ', cleaned_english) 

    # 5. 수량, 측정, 단위 정보 제거
    cleaned_english = re.sub(r'\d+[a-z]{1,3}|not more than|exceeding', '', cleaned_english, flags=re.IGNORECASE)
    
    # 6. 최종 정리
    final_text = re.sub(r'\s+', ' ', cleaned_english).strip()
    
    return final_text

def extract_pos_keywords(description: str) -> List[str]:
    # (생략: 기존 extract_pos_keywords 함수)
    if not description: return []
    words = description.lower().split()
    # NLTK PerceptronTagger가 사용됩니다. (LookupError 해결 필수)
    tagged_words = nltk.tag.pos_tag(words) 
    allowed_tags = {'NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS'}
    keywords = [
        word for word, tag in tagged_words 
        if tag in allowed_tags and len(word) > 2 and word.isalpha()
    ]
    hs_stop_words = {
        'parts', 'pieces', 'thereof', 'subheading', 'chapter', 'heading',
        'level', 'mixtures', 'whether', 'known', 'having', 'other', 
        'used', 'products', 'articles', 'produce', 'raw', 'made', 
        'containing', 'total', 'certain', 'various', 'net', 'edible',
        'goods', 'live', 'fresh', 'frozen', 'dried', 'animals', 'obtained' 
    }
    filtered_keywords = [word for word in keywords if word not in hs_stop_words]
    return list(set(filtered_keywords))

# --- 2. 메인 Category 채우기 함수 ---

def fill_empty_categories_recursive(df_master: pd.DataFrame) -> pd.DataFrame:
    """
    Code 길이 순으로 정렬하여, category가 비어있는 Detail 항목을 부모 category로 채웁니다.
    최종 결과는 쉼표로 구분된 문자열(대괄호 없음)로 저장됩니다.
    """
    df = df_master.copy()
    
    # 1. Keywords 컬럼을 실제 리스트 타입으로 변환 (오류 방지)
    # 'category' 컬럼을 사용하고, 쉼표로 구분된 문자열을 리스트로 변환
    def string_to_list(s):
        if pd.isna(s) or s in ('', '[]', 'list()'): return []
        if isinstance(s, list): return s
        if isinstance(s, str) and s.startswith('['):
             try: return ast.literal_eval(s)
             except: return [item.strip().lower() for item in s.strip('[]').split(',') if item.strip()]
        if isinstance(s, str):
            # 쉼표로 구분된 일반 문자열을 리스트로 변환
            return [item.strip().lower() for item in s.split(',') if item.strip()]
        return []

    df['category_list'] = df['category'].apply(string_to_list)
    
    # 2. Code 길이 및 문자열 오름차순 정렬 (순차적 상속의 핵심!)
    df['code_len'] = df['code'].str.len()
    df = df.sort_values(by=['code_len', 'code'], ascending=True).reset_index(drop=True)
    
    print(f"--- Category 순차 상속 시작: 총 {len(df)}개 항목 ---")
    
    # df를 dictionary로 변환하여 부모의 정보를 빠르게 참조
    code_to_index = {code: i for i, code in enumerate(df['code'])}

    # 3. 순차적 상속 루프 (df.index 순회)
    for index in df.index:
        row = df.loc[index]
        current_keywords = row['category_list']
        
        # 3.1. 이미 채워진 항목은 건너뜁니다.
        if current_keywords:
            continue
            
        parent_code = row['parent_code']
        
        # 3.2. 부모 Code가 없거나(루트) 처리 대상(Detail)이 아닌 경우 건너뜁니다.
        if pd.isna(parent_code) or row['source'] != 'DETAIL':
            continue

        # --- A. 부모 Category 획득 ---
        parent_keywords = []
        if parent_code in code_to_index:
            parent_idx = code_to_index[parent_code]
            # 부모는 이미 처리되었으므로, 그 결과(category_list)를 가져옵니다.
            parent_keywords = df.at[parent_idx, 'category_list'] 

        # 3.3. 추출 및 상속 로직
        
        english_description = clean_text_for_pos(row['description']) 
        
        # 'Other' 조건 확인 (Category 상속 기준)
        is_just_other = english_description.lower().strip() in ('other', '기타', 'etc', '')

        if is_just_other:
            # 1. 'Other'인 경우: 부모의 Category를 그대로 상속
            final_keywords = parent_keywords
            
        else:
            # 2. 내용이 있는 경우: 영어 명사 추출 + 부모 Category 병합
            extracted_keywords = extract_pos_keywords(english_description)
            
            final_keywords = set(parent_keywords)
            final_keywords.update(extracted_keywords)
            final_keywords = list(final_keywords)
        
        # 🚨 [새로운 로직 추가]: 상태 및 처리 키워드 보강
        state_keywords = extract_state_keywords(english_description)
        
        if state_keywords:
            final_keywords = set(final_keywords)
            final_keywords.update(state_keywords)
            final_keywords = list(final_keywords)
        # ----------------------------------------------
        
        # 3.4. 데이터프레임 업데이트
        df.at[index, 'category_list'] = final_keywords
        df.at[index, 'category'] = ", ".join(final_keywords)

    print("✅ Category 순차 상속 및 채우기 완료.")
    # 최종 결과에서 리스트 저장용 컬럼과 임시 컬럼을 제거하고 출력
    return df.drop(columns=['category_list', 'code_len'])

# --- 최종 실행 블록 ---

# ⚠️ df_master 데이터프레임이 이미 정의되어 있어야 합니다.

df_master_final = fill_empty_categories_recursive(df_master)
# 결과 확인
display(df_master_final[df_master_final['source'] == 'DETAIL'].head(10))

--- Category 순차 상속 시작: 총 19407개 항목 ---
✅ Category 순차 상속 및 채우기 완료.


,code,description,parent_code,source,category
6940,0207603,"기타(신선한 것이나 냉장한 것으로 한정한다) (Other, fresh or chil...",020760,DETAIL,"meat, offal, chilled, fresh"
6941,0207604,"기타(냉동한 것으로 한정한다) (Other, frozen)",020760,DETAIL,"meat, offal, frozen"
6942,0307992,건조한 것 (Dried),030799,DETAIL,"molluscs, dried"
6943,0307994,훈제한 것 (Smoked ),030799,DETAIL,"smoked, molluscs"
6944,0404102,변성유장 (Modified whey),040410,DETAIL,"dairy, whey"
6945,0507901,녹용과 녹각[가루와 웨이스트(waste)를 포함한다] (Young antlers a...,050790,DETAIL,"waste, animal, antlers, powder, young"
6946,1211201,"인삼 뿌리(신선ㆍ냉장하거나 건조한 것) (Ginseng root, fresh, ch...",121120,DETAIL,"similar, root, dried, ginseng, perfumery, root..."
6947,1211202,"인삼 가루(신선ㆍ 냉장하거나 건조한 것) (Ginseng powder, fresh,...",121120,DETAIL,"similar, dried, powder, perfumery, roots, gins..."
6948,1211203,인삼잎과 줄기ㆍ종자(신선ㆍ 냉장하거나 건조한 것) (Leaves and stems ...,121120,DETAIL,"similar, stems, leaves, dried, ginseng, perfum..."
6949,1211209,기타 (Other),121120,DETAIL,"similar, ginseng, perfumery, roots, purposes, ..."


In [27]:
df_master_final = df_master_final.sort_values('code').reset_index(drop=True)
df_master_final

,code,description,parent_code,source,category
0,01,Animals; live,NaN,STRUCTURE,commodities
1,0101,"Horses, asses, mules and hinnies; live",01,STRUCTURE,"mules, hinnies, horses"
2,010121,"Horses; live, pure-bred breeding animals",0101,STRUCTURE,horses
3,0101211000,농가 사육용 (For farm breeding),010121,DETAIL,말
4,0101219000,기타 (Other),010121,DETAIL,말
...,...,...,...,...,...
19402,9706903000,기타 (Other),970690,DETAIL,수집품 및 골동품
19403,99,Commodities not specified according to kind,NaN,STRUCTURE,"kind, commodities"
19404,9999,Commodities not specified according to kind,99,STRUCTURE,"kind, commodities"
19405,999999,Commodities not specified according to kind,9999,STRUCTURE,"kind, commodities"


In [28]:
df_master_final.drop(19406, inplace=True)
df_master_final

,code,description,parent_code,source,category
0,01,Animals; live,NaN,STRUCTURE,commodities
1,0101,"Horses, asses, mules and hinnies; live",01,STRUCTURE,"mules, hinnies, horses"
2,010121,"Horses; live, pure-bred breeding animals",0101,STRUCTURE,horses
3,0101211000,농가 사육용 (For farm breeding),010121,DETAIL,말
4,0101219000,기타 (Other),010121,DETAIL,말
...,...,...,...,...,...
19401,9706902000,악기류 (Musical instruments),970690,DETAIL,수집품 및 골동품
19402,9706903000,기타 (Other),970690,DETAIL,수집품 및 골동품
19403,99,Commodities not specified according to kind,NaN,STRUCTURE,"kind, commodities"
19404,9999,Commodities not specified according to kind,99,STRUCTURE,"kind, commodities"


In [29]:
df_detail_clean.loc[df_detail_clean['code'] == '0307992']

,code,description,parent_code,category,source
650,0307992,건조한 것 (Dried),030799,NaN,DETAIL


In [30]:
df_master_final.loc[df_master_final['code'] == '0307992']

,code,description,parent_code,source,category
991,0307992,건조한 것 (Dried),030799,DETAIL,"molluscs, dried"


In [31]:
df_master_final.drop(columns='source', inplace=True)

In [32]:
df_master_final

,code,description,parent_code,category
0,01,Animals; live,NaN,commodities
1,0101,"Horses, asses, mules and hinnies; live",01,"mules, hinnies, horses"
2,010121,"Horses; live, pure-bred breeding animals",0101,horses
3,0101211000,농가 사육용 (For farm breeding),010121,말
4,0101219000,기타 (Other),010121,말
...,...,...,...,...
19401,9706902000,악기류 (Musical instruments),970690,수집품 및 골동품
19402,9706903000,기타 (Other),970690,수집품 및 골동품
19403,99,Commodities not specified according to kind,NaN,"kind, commodities"
19404,9999,Commodities not specified according to kind,99,"kind, commodities"


In [33]:
df_master_final.to_csv('all_hscode.csv', index=False)